# [HexSim](http://www.hexsim.net/) visualiztions

* reproject a shapefile w/ geopandas
* write out geojson
* use styling to improve output

In [ ]:
from IPython.display import display
import geopandas as gpd
hexes = gpd.read_file('data/hexsim/hexes2.shp')

In [ ]:
display(hexes.head())
% matplotlib inline
hexes.plot()

In [ ]:
print(hexes.crs)
hexes.to_crs({'init':'epsg:4326'}, inplace=True)
print(hexes.crs)
hexes.to_file('data/hexsim/hexes2.json', driver='GeoJSON')

In [ ]:
extent = hexes.unary_union.bounds
lat_Center = (extent[1] + extent[3]) / 2
lon_Center = (extent[0] + extent[2]) / 2
print(lat_Center,lon_Center)

In [ ]:
import pandas as pd
heat = pd.read_csv('data/hexsim/hex_temp.csv')
heat.drop(['CWR_ID','SIZE_RANK'],axis=1,inplace=True)
heat.head()

In [ ]:
dep = pd.read_csv('data/hexsim/reach1_500_depth_m.csv')
len(dep)

In [ ]:
hexes = hexes.merge(dep, on='Hex_ID')
hexes = hexes.merge(heat, on='Hex_ID')

In [ ]:
import folium
from folium import plugins

hex_map = folium.Map(location=[lat_Center,lon_Center], 
                               tiles="Stamen Terrain", 
                               zoom_start=15)

geojson = folium.GeoJson('./data/hexsim/hexes2.json', 
                            name='Hexes')

hex_map.add_child(geojson)
folium.LayerControl().add_to(hex_map)
plugins.FloatImage('https://debboutr.github.io/epa_clipped.png', bottom=74, left=8).add_to(hex_map)

hex_map

In [ ]:
m = folium.Map(
    location=[lat_Center,lon_Center],
    tiles='Stamen Terrain',
    zoom_start=15)

m.choropleth(
    geo_data='./data/hexsim/hexes2.json',
    data=pd.read_csv('./data/hexsim/reach1_500_depth_m.csv'),
    columns=['Hex_ID','Depth_m'],
    key_on='feature.properties.Hex_ID',
    name='Hexes',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
#     line_color='white',
#     highlight=True,
    legend_name='Depth (m)')

folium.LayerControl().add_to(m)
# m.save('./maps/hex_depth.html')
m

In [ ]:
hex_map = folium.Map(location=[lat_Center,lon_Center], 
                               tiles=("https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/256/{z}/{x}/{y}?"
                                      "access_token=pk.eyJ1IjoiZGFyYW9iZWlybmUiLCJhIjoidlV2WXZKRSJ9."
                                      "EwVOY6b8UeyIz3N2UAu5pw"), 
                               zoom_start=14, attr='@Mapbox')

json = folium.GeoJson('./data/hexsim/hexes2.json', 
                      name='Hexes',
                      style_function=lambda x: {     'fillColor' :'red', 
                                                  'fill_opacity' : 0.47, 
                                                         'color' : 'green',
                                                        'weight' : 1},
                      highlight_function= lambda x: {'fillColor' : 'blue',
#                                                          'color' : 'yellow',
                                                        'weight' : 3})

hex_map.add_child(json)

html_code = ('<style>'
         '#title{'
             'position: fixed;'
             'bottom: 50px;'
             'left: 50px;'
             'width:33%;'
             'z-index:9999;'
             'background-color: #D3D3D3;'
             'border-radius: 25px;'
             'border: 3px solid #000000;'
             'opacity: 0.74;'
         '}'
         '.center {'
            'display: block;'
            'margin-left: auto;'
            'margin-right: auto;'

         '}'
     '</style>'     
     '<div id="title">'
        '<br>'     
        '<img src="https://debboutr.github.io/epa_small.png" alt="no_bueno" class="center">'
        '<a href="http://www.hexsim.net/" target="_blank"><h3 style="color:#000000; text-align: center;">Hex Sim</h3></a>'
        '<p style="text-align:center;">HexSim is a free, versatile, multi-species, life history simulator ideal for building models '
             'of animal and plant population viability, interactions, and responses to disturbance. HexSim '
             'models are spatially-explicit and individual-based, and HexSim individuals can be assigned '
             'dynamic life history traits. HexSim also includes a full genetics sub-model, making it a '
             'true eco-evo simulator. </p>'
     '</div>')

elem = folium.Element(html_code)
hex_map.get_root().html.add_child(elem)

# hex_map.save("watermap.html")
hex_map